<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Artificial Intelligence in Finance

## Reinforcement Learning

&copy; Dr Yves J Hilpisch | The Python Quants GmbH

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
import os
import math
import random
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=4, suppress=True)
os.environ['PYTHONHASHSEED'] = '0'

## `CartPole` Environment 

In [ ]:
import gym

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:

env.action_space.seed(100)

In [ ]:
#状态空间，分别为 推车位置、推车速度、极角、极角速度
env.observation_space

In [ ]:
env.observation_space.low.astype(np.float16)

In [ ]:
env.observation_space.high.astype(np.float16)

In [ ]:
state = env.reset(seed=100)

In [ ]:
state

In [ ]:
env.action_space

In [ ]:
env.action_space.n

In [ ]:
env.action_space.sample()

In [ ]:
env.action_space.sample() 

In [ ]:
a = env.action_space.sample()
a

In [ ]:
state, reward, done,truncated, info = env.step(a)
state, reward, done,truncated, info

In [ ]:
state = env.reset(seed=100)
for e in range(1, 200):
    a = env.action_space.sample()
    state, reward, done,truncated, info = env.step(a)
    print(f'step={e:2d} | state={state} | action={a} | reward={reward}')
    if done and (e + 1) < 200:
        print('*** FAILED ***')
        break

In [ ]:
done

## Dimensionality Reduction

See http://kvfrans.com/simple-algoritms-for-solving-cartpole/.

In [ ]:
np.random.seed(100)

In [ ]:
weights = np.random.random(4) * 2 - 1

In [ ]:
weights

In [ ]:
state = env.reset()

In [ ]:
state

In [ ]:
s = np.dot(state[0], weights)
s

## Action Rule

In [ ]:
if s < 0:
    a = 0
else:
    a = 1

In [ ]:
a

## Total Reward per Episode

In [ ]:
def run_episode(env, weights):  
    state = env.reset()[0]
    treward = 0
    for _ in range(200):
        s = np.dot(state, weights)
        a = 0 if s < 0 else 1
        state, reward, done,truncated, info = env.step(a)
        treward += reward
        if done:
            break
    return treward


In [ ]:
run_episode(env, weights)


## Simple Learning 

In [ ]:
env = gym.make('CartPole-v0')

def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    env.reset(seed=seed)


In [ ]:
set_seeds()
num_episodes = 1000

In [ ]:
besttreward = 0
for e in range(1, num_episodes + 1):
    weights = np.random.rand(4) * 2 - 1
    treward = run_episode(env, weights)
    if treward > besttreward:
        besttreward = treward
        bestweights = weights
        if treward == 200:
            print(f'SUCCESS | episode={e}')
            break
        print(f'UPDATE  | episode={e}')

In [ ]:
weights

## Testing the Results

In [ ]:
res = []
for _ in range(100):
    treward = run_episode(env, weights)
    res.append(treward)
res[:10]

In [ ]:
sum(res) / len(res)

## DNN Learning

In [ ]:
import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from tensorflow.python.framework.ops import enable_eager_execution

In [ ]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from sklearn.metrics import accuracy_score

In [ ]:
def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    env.reset(seed=seed)
    env.action_space.seed(seed)

In [180]:
class NNAgent:
    def __init__(self):
        self.max = 0
        self.scores = list()
        self.memory = list()
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=4,
                        activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        constant = tf.constant(0.001, dtype=tf.float32)
        model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.legacy.RMSprop())

        return model
        
    def act(self, state):
        if random.random() <= 0.5:
            return env.action_space.sample()
        action = np.where(self.model.predict(
            state, batch_size=None)[0, 0] > 0.5, 1, 0)
        return action
                    
    def train_model(self, state, action):
        self.model.fit(state, np.array([action,]),
                       epochs=1, verbose=False)
    
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            state = env.reset(seed=100)[0]
            for _ in range(201):
                state = np.reshape(state, [1, 4])
                action = self.act(state)
                next_state, reward, done,truncated, info = env.step(action)
                if done:
                    score = _ + 1
                    self.scores.append(score)
                    self.max = max(score, self.max)
                    print('episode: {:4d}/{} | score: {:3d} | max: {:3d}'
                          .format(e, episodes, score, self.max), end='\r')
                    break
                self.memory.append((state, action))
                self.train_model(state, action)
                state = next_state

set_seeds(100)
agent = NNAgent()

In [178]:
episodes = 1000

In [181]:
agent.learn(episodes)

AssertionError: 

In [ ]:
sum(agent.scores) / len(agent.scores)

In [ ]:
f = np.array([m[0][0] for m in agent.memory])
f

In [ ]:
l = np.array([m[1] for m in agent.memory])
l

In [ ]:
accuracy_score(np.where(agent.model.predict(f) > 0.5, 1, 0), l)

## Q Learning

See https://keon.io/deep-q-learning/

In [ ]:
from collections import deque
from keras.optimizers import Adam, RMSprop

In [ ]:
class DQLAgent:
    def __init__(self, gamma=0.95, hu=24, opt=Adam,
           lr=0.001, finish=False):
        self.finish = finish
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.gamma = gamma
        self.batch_size = 32
        self.max_treward = 0
        self.averages = list()
        self.memory = deque(maxlen=2000)
        self.osn = env.observation_space.shape[0]
        self.model = self._build_model(hu, opt, lr)
        
    def _build_model(self, hu, opt, lr):
        model = Sequential()
        model.add(Dense(hu, input_dim=self.osn,
                        activation='relu'))
        model.add(Dense(hu, activation='relu'))
        model.add(Dense(env.action_space.n, activation='linear'))
        model.compile(loss='mse', optimizer=opt(lr=lr))
        return model
        
    def act(self, state):
        if random.random() <= self.epsilon:
            return env.action_space.sample()
        action = self.model.predict(state)[0]
        return np.argmax(action)
    
    def replay(self):
        batch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in batch:
            if not done:
                reward += self.gamma * np.amax(
                    self.model.predict(next_state)[0])
            target = self.model.predict(state)
            target[0, action] = reward
            self.model.fit(state, target, epochs=1,
                           verbose=False)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def learn(self, episodes):
        trewards = []
        for e in range(1, episodes + 1):
            state = env.reset()
            state = np.reshape(state, [1, self.osn])
            for _ in range(5000):
                action = self.act(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state,
                                        [1, self.osn])
                self.memory.append([state, action, reward,
                                     next_state, done])
                state = next_state
                if done:
                    treward = _ + 1
                    trewards.append(treward)
                    av = sum(trewards[-25:]) / 25
                    self.averages.append(av)
                    self.max_treward = max(self.max_treward, treward)
                    templ = 'episode: {:4d}/{} | treward: {:4d} | '
                    templ += 'av: {:6.1f} | max: {:4d}'
                    print(templ.format(e, episodes, treward, av,
                                       self.max_treward), end='\r')
                    break
            if av > 195 and self.finish:
                print()
                break
            if len(self.memory) > self.batch_size:
                self.replay()
    def test(self, episodes):
        trewards = []
        for e in range(1, episodes + 1):
            state = env.reset()
            for _ in range(5001):
                state = np.reshape(state, [1, self.osn])
                action = np.argmax(self.model.predict(state)[0])
                next_state, reward, done, info = env.step(action)
                state = next_state
                if done:
                    treward = _ + 1
                    trewards.append(treward)
                    print('episode: {:4d}/{} | treward: {:4d}'
                          .format(e, episodes, treward), end='\r')
                    break
        return trewards

In [ ]:
episodes = 1000

In [ ]:
set_seeds(100)
agent = DQLAgent(finish=True)

In [ ]:
%time agent.learn(episodes)

In [ ]:
plt.figure(figsize=(10, 6))
x = range(len(agent.averages))
y = np.polyval(np.polyfit(x, agent.averages, deg=3), x)
plt.plot(agent.averages, label='moving average')
plt.plot(x, y, 'r--', label='trend')
plt.xlabel('episodes')
plt.ylabel('total reward')
plt.legend();

In [ ]:
trewards = agent.test(100)

In [ ]:
sum(trewards) / len(trewards)

## Finance Environment

In [ ]:
class observation_space:
    def __init__(self, n):
        self.shape = (n,)

In [ ]:
class action_space:
    def __init__(self, n):
        self.n = n
    def seed(self, seed):
        pass
    def sample(self):
        return random.randint(0, self.n - 1)

In [ ]:
class Finance:
    url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
    def __init__(self, symbol, features):
        self.symbol = symbol
        self.features = features
        self.observation_space = observation_space(4)
        self.osn = self.observation_space.shape[0]
        self.action_space = action_space(2)
        self.min_accuracy = 0.475
        self._get_data()
        self._prepare_data()
    def _get_data(self):
        self.raw = pd.read_csv(self.url, index_col=0,
                               parse_dates=True).dropna()
    def _prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        self.data = (self.data - self.data.mean()) / self.data.std()
        self.data['d'] = np.where(self.data['r'] > 0, 1, 0)
    def _get_state(self):
        return self.data[self.features].iloc[
            self.bar - self.osn:self.bar].values
    def seed(self, seed=None):
        pass
    def reset(self):
        self.treward = 0
        self.accuracy = 0
        self.bar = self.osn
        state = self.data[self.features].iloc[
            self.bar - self.osn:self.bar]
        return state.values
    def step(self, action):
        correct = action == self.data['d'].iloc[self.bar]
        reward = 1 if correct else 0
        self.treward += reward
        self.bar += 1
        self.accuracy = self.treward / (self.bar - self.osn)
        if self.bar >= len(self.data):
            done = True
        elif reward == 1:
            done = False
        elif (self.accuracy < self.min_accuracy and
              self.bar > self.osn + 10):
            done = True
        else:
            done = False
        state = self._get_state()
        info = {}
        return state, reward, done, info

In [ ]:
env = Finance('EUR=', 'EUR=')

In [ ]:
env.reset()

In [ ]:
a = env.action_space.sample()
a

In [ ]:
env.step(a)

In [ ]:
set_seeds(100)
agent = DQLAgent(gamma=0.5, opt=RMSprop)

In [ ]:
episodes = 1000

In [ ]:
%time agent.learn(episodes)

In [ ]:
agent.test(3)

In [ ]:
plt.figure(figsize=(10, 6))
x = range(len(agent.averages))
y = np.polyval(np.polyfit(x, agent.averages, deg=3), x)
plt.plot(agent.averages, label='moving average')
plt.plot(x, y, 'r--', label='regression')
plt.xlabel('episodes')
plt.ylabel('total reward')
plt.legend();

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>